In [1]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, adjusted_rand_score

In [2]:
# Step 1: Load Subdatasets
def load_subdataset(file_path):
    return pd.read_csv(file_path)

data = load_subdataset('madelon_66.csv')
data

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,target
0,0.390,0.506,0.511,0.364,0.665,0.304,0.347,0.455,0.436,0.66,...,0.324,0.539,0.206,0.709,0.734,0.535,0.55,0.459,0.495,-1
1,0.585,0.815,0.641,0.636,0.274,0.630,0.426,0.727,0.154,0.44,...,0.441,0.317,0.411,0.543,0.383,0.442,0.38,0.320,0.774,-1
2,0.585,0.575,0.609,0.515,0.439,0.391,0.610,0.727,0.410,0.48,...,0.412,0.500,0.453,0.603,0.448,0.488,0.62,0.488,0.526,-1
3,0.463,0.588,0.430,0.455,0.692,0.761,0.347,0.727,0.470,0.44,...,0.529,0.122,0.508,0.138,0.477,0.442,0.73,0.523,0.558,-1
4,0.341,0.511,0.673,0.424,0.415,0.196,0.675,0.455,0.487,0.12,...,0.441,0.444,0.509,0.360,0.555,0.488,0.61,0.616,0.426,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711,0.610,0.657,0.789,0.348,0.503,0.304,0.357,0.455,0.504,0.38,...,0.500,0.511,0.264,0.543,0.367,0.488,0.32,0.733,0.537,1
1712,0.561,0.532,0.532,0.606,0.457,0.478,0.819,0.455,0.556,0.40,...,0.471,0.372,0.343,0.672,0.597,0.395,0.54,0.527,0.547,1
1713,0.415,0.481,0.447,0.485,0.582,0.391,0.588,0.455,0.556,0.52,...,0.500,0.650,0.509,0.607,0.393,0.512,0.49,0.488,0.521,1
1714,0.561,0.532,0.444,0.424,0.607,0.565,0.567,0.273,0.573,0.56,...,0.500,0.556,0.344,0.555,0.562,0.302,0.44,0.516,0.532,1


In [3]:
true_labels = data["target"].tolist()
unique_labels = set(true_labels)
print(unique_labels)

{1, -1}


In [4]:
data_cleaned = data.drop(["Index","target"], axis=1)
data_cleaned

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0.390,0.506,0.511,0.364,0.665,0.304,0.347,0.455,0.436,0.66,...,0.479,0.324,0.539,0.206,0.709,0.734,0.535,0.55,0.459,0.495
1,0.585,0.815,0.641,0.636,0.274,0.630,0.426,0.727,0.154,0.44,...,0.352,0.441,0.317,0.411,0.543,0.383,0.442,0.38,0.320,0.774
2,0.585,0.575,0.609,0.515,0.439,0.391,0.610,0.727,0.410,0.48,...,0.324,0.412,0.500,0.453,0.603,0.448,0.488,0.62,0.488,0.526
3,0.463,0.588,0.430,0.455,0.692,0.761,0.347,0.727,0.470,0.44,...,0.261,0.529,0.122,0.508,0.138,0.477,0.442,0.73,0.523,0.558
4,0.341,0.511,0.673,0.424,0.415,0.196,0.675,0.455,0.487,0.12,...,0.500,0.441,0.444,0.509,0.360,0.555,0.488,0.61,0.616,0.426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711,0.610,0.657,0.789,0.348,0.503,0.304,0.357,0.455,0.504,0.38,...,0.535,0.500,0.511,0.264,0.543,0.367,0.488,0.32,0.733,0.537
1712,0.561,0.532,0.532,0.606,0.457,0.478,0.819,0.455,0.556,0.40,...,0.627,0.471,0.372,0.343,0.672,0.597,0.395,0.54,0.527,0.547
1713,0.415,0.481,0.447,0.485,0.582,0.391,0.588,0.455,0.556,0.52,...,0.451,0.500,0.650,0.509,0.607,0.393,0.512,0.49,0.488,0.521
1714,0.561,0.532,0.444,0.424,0.607,0.565,0.567,0.273,0.573,0.56,...,0.542,0.500,0.556,0.344,0.555,0.562,0.302,0.44,0.516,0.532


In [5]:
def spectral_clustering(dataframe, labels, similarity_graph, laplacian, number_of_clusters, local_sigma = None, epsilon = None, k_knn = None, k_mknn = None):

    # Pairwise distances
    dimension = dataframe.shape[0]
    dist_mat = squareform(pdist(dataframe))

    if similarity_graph == "full":

        #calculate local sigma
        sigmas = np.zeros(dimension)
        for i in tqdm(range(len(dist_mat))):
            sigmas[i] = sorted(dist_mat[i])[local_sigma]

        # Adjaceny matrix with optimal sigma
        adjacency_matrix = np.zeros([dimension, dimension])
        for i in tqdm(range(dimension)):
            for j in range(i+1, dimension):
                d = np.exp(-1*dist_mat[i,j]**2/(sigmas[i]*sigmas[j]))
                adjacency_matrix[i,j] = d
                adjacency_matrix[j,i] = d


    elif similarity_graph == "eps":

        # Adjaceny matrix with epsilon threshold
        adjacency_matrix = np.zeros([dimension, dimension])

        for i in tqdm(range(dimension)):
            for j in range(i+1, dimension):
                if dist_mat[i,j] < epsilon:
                    d = 1
                else:
                    d = 0
                adjacency_matrix[i,j] = d
                adjacency_matrix[j,i] = d


    elif similarity_graph == "knn":

        # Adjaceny matrix with k-neighbours
        adjacency_matrix = np.zeros([dimension, dimension])

        for i in tqdm(range(dimension)):
            # Sort distances for node i and get indices of the k nearest neighbors
            sorted_indices = np.argsort(dist_mat[i])
            k_nearest_indices = sorted_indices[1:k_knn+1]  # Exclude the node itself

            # Update the adjacency matrix
            adjacency_matrix[i, k_nearest_indices] = 1


    else:

        # Adjaceny matrix with mutual k-neighbours
        adjacency_matrix = np.zeros([dimension, dimension])

        for i in tqdm(range(dimension)):
            # Sort distances for node i and get indices of the k nearest neighbors
            sorted_indices = np.argsort(dist_mat[i])
            k_nearest_indices = sorted_indices[1:k_mknn+1]  # Exclude the node itself

            for neighbor in k_nearest_indices:
                # Check if node i is also among the k-nearest neighbors of the current neighbor
                neighbor_sorted_indices = np.argsort(dist_mat[neighbor])
                if i in neighbor_sorted_indices[1:k_mknn+1]:
                    # Connect nodes if they are mutual k-nearest neighbors
                    adjacency_matrix[i, neighbor] = 1
                    adjacency_matrix[neighbor, i] = 1

    # Calculate degree matrix
    degrees = np.sum(adjacency_matrix, axis=1)
    degree_matrix = np.diag(degrees)

    if laplacian == "sym":

        # Normalized Symmetric laplacian matrix
        d_inv_sqrt = np.zeros_like(degrees)
        nonzero = degrees > 0
        d_inv_sqrt[nonzero] = 1.0 / np.sqrt(degrees[nonzero])
        d_half = np.diag(d_inv_sqrt)
        laplacian_matrix_normalized = d_half @ adjacency_matrix @ d_half

    if laplacian == "rw":

        # Normalized Random Walk laplacian matrix
        d_inv = np.zeros_like(degrees)
        nonzero = degrees > 0
        d_inv[nonzero] = 1.0 / degrees[nonzero]
        d_inverse = np.diag(d_inv)
        laplacian_matrix_normalized = d_inverse @ adjacency_matrix

    if laplacian == "ad":

        # Adaptive Laplacian matrix
        D_local = np.zeros_like(degrees)
        for i in range(len(degrees)):
            neighbors = np.where(adjacency_matrix[i] > 0)[0]
            if len(neighbors) > 0 and degrees[i] > 0:
                D_local[i] = np.sum(degrees[neighbors]) / degrees[i]
            else:
                D_local[i] = 0
        D_local_inv_sqrt = np.zeros_like(D_local)
        nonzero = D_local > 0
        D_local_inv_sqrt[nonzero] = 1.0 / np.sqrt(D_local[nonzero])
        D_local_inv = np.diag(D_local_inv_sqrt)
        laplacian_matrix_normalized = D_local_inv @ adjacency_matrix @ D_local_inv

    if check_symmetric(laplacian_matrix_normalized) :
        # Calculating eigenvalues and eigenvectors for symmetric matrix
        e, v = np.linalg.eigh(laplacian_matrix_normalized)
    else:
        # Calculating eigenvalues and eigenvectors for non-symmetric matrix
        e, v = np.linalg.eig(laplacian_matrix_normalized)
        idx = np.argsort(np.real(e))
        e = np.real(e[idx])
        v = np.real(v[:, idx])

    # Calculate eigengap
    eigengap = np.diff(e)
    optimal_number_of_clusters = np.argmax(eigengap[:10]) + 1

    if number_of_clusters != None:
        # First case: k
        n_clusters = max(number_of_clusters,2)
    else:
        # Second case: optimal number of clusters from eigengap
        n_clusters = max(optimal_number_of_clusters,2)

    results = []
    
    # adj_filename, laplacian_filename, X_filename = save_matrices(similarity_graph,laplacian, adjacency_matrix, laplacian_matrix_normalized, X)

    # KMeans clustering
    X = v[:, -n_clusters:]
    clustering = KMeans(n_clusters=n_clusters, random_state=42, n_init=100)
    cluster_labels = clustering.fit_predict(X)

    # Calculate evaluation metrics
    sil_score = silhouette_score(dataframe, cluster_labels)
    ar_score = adjusted_rand_score(labels, cluster_labels)

    results.append((sil_score, ar_score, n_clusters,cluster_labels))
    # results.append((sil_score, ar_score, n_clusters,cluster_labels, adj_filename, laplacian_filename, X_filename))

    return results

In [6]:
def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

In [7]:
# Function to calculate dynamic ranges based on data size and pairwise distances
def get_dynamic_search_space(data):
    # Number of points in the dataset
    n = data.shape[0]

    # Compute pairwise distances
    dist_mat = squareform(pdist(data))
    flat_distances = dist_mat[np.tril_indices(n, -1)]

    # Dynamic range for local_sigma (based on square root of n)
    local_sigma_min = max(1, int(np.sqrt(n) / 2))
    local_sigma_max = int(np.sqrt(n))
    
    # Dynamic range for epsilon (based on distance percentiles)
    epsilon_min = np.percentile(flat_distances, 80)  # 80th percentile
    epsilon_max = np.percentile(flat_distances, 95)  # 95th percentile
    
    # Dynamic range for k (based on number of data points)
    k_min = max(5, int(0.01 * n))  # 1% of dataset size, but at least 5
    k_max = min(int(0.2 * n), n - 1)  # 20% of dataset size, but never more than n-1
    
    # Ensure k_min does not exceed k_max
    if k_min > k_max:
        k_min = max(5, int(0.01 * n))  # Keep dynamic range based on percentage but within limits
    
    return (local_sigma_min, local_sigma_max), (epsilon_min, epsilon_max), (k_min, k_max)

# Optimization functions for each parameter

# Optimize local_sigma for "full" graph
def optimize_local_sigma(data, labels, laplacians, number_of_clusters):
    (local_sigma_min, local_sigma_max), _, _ = get_dynamic_search_space(data)

    def objective_local_sigma(local_sigma):
        silhouette_scores = []
        local_sigma = int(local_sigma[0])
        try:
            for laplacian in laplacians:
                results = spectral_clustering(data, labels, similarity_graph="full", laplacian=laplacian, number_of_clusters=number_of_clusters, local_sigma=local_sigma)
                silhouette_scores.append(results[0][0])
            return -np.mean(silhouette_scores)
        except (ValueError, np.linalg.LinAlgError) as e:
            print(f"Skipping local_sigma={local_sigma} due to error: {e}")
            return 1e6  # Return a large value to penalize the failed set of hyperparameters

    result = gp_minimize(objective_local_sigma, [(local_sigma_min, local_sigma_max)], n_calls=20, n_random_starts=10, random_state=42)

    if result.fun < 1e6:
        best_local_sigma = result.x[0]
        print(f"Best local sigma: {best_local_sigma}")
        return result
    else:
        print("No valid local_sigma found.")
        return None


# Optimize epsilon for "eps" graph
def optimize_epsilon(data, labels, laplacians, number_of_clusters):
    _, (epsilon_min, epsilon_max), _ = get_dynamic_search_space(data)

    def objective_epsilon(epsilon):
        silhouette_scores = []
        epsilon = float(epsilon[0])
        try:
            for laplacian in laplacians:
                results = spectral_clustering(data, labels, similarity_graph="eps", laplacian=laplacian, number_of_clusters=number_of_clusters, epsilon=epsilon)
                silhouette_scores.append(results[0][0])
            return -np.mean(silhouette_scores)
        except (ValueError, np.linalg.LinAlgError) as e:
            print(f"Skipping epsilon={epsilon} due to error: {e}")
            return 1e6  # Return a large value to penalize the failed set of hyperparameters

    result = gp_minimize(objective_epsilon, [(epsilon_min, epsilon_max)], n_calls=20, n_random_starts=10, random_state=42)

    if result.fun < 1e6:
        epsilon = result.x[0]
        print(f"Best epsilon: {epsilon}")
        return result
    else:
        print("No valid epsilon found.")
        return None


# Optimize k for "knn" graph
def optimize_k_knn(data, labels, laplacians, number_of_clusters):
    _, _, (k_min, k_max) = get_dynamic_search_space(data)

    def objective_k_knn(k):
        silhouette_scores = []
        k = int(k[0])
        try:
            for laplacian in laplacians:
                results = spectral_clustering(data, labels, similarity_graph="knn", laplacian=laplacian, number_of_clusters=number_of_clusters, k_knn=k)
                silhouette_scores.append(results[0][0])
            return -np.mean(silhouette_scores)
        except (ValueError, np.linalg.LinAlgError) as e:
            print(f"Skipping k={k} due to error: {e}")
            return 1e6  # Return a large value to penalize the failed set of hyperparameters

    result = gp_minimize(objective_k_knn, [(k_min, k_max)], n_calls=20, n_random_starts=10, random_state=42)

    if result.fun < 1e6:
        k_knn = result.x[0]
        print(f"Best k for knn: {k_knn}")
        return result
    else:
        print("No valid k for knn found.")
        return None


# Optimize k for "mknn" graph
def optimize_k_mknn(data, labels, laplacians, number_of_clusters):
    _, _, (k_min, k_max) = get_dynamic_search_space(data)

    def objective_k_mknn(k):
        silhouette_scores = []
        k = int(k[0])
        try:
            for laplacian in laplacians:
                results = spectral_clustering(data, labels, similarity_graph="mknn", laplacian=laplacian, number_of_clusters=number_of_clusters, k_mknn=k)
                silhouette_scores.append(results[0][0])
            return -np.mean(silhouette_scores)
        except (ValueError, np.linalg.LinAlgError) as e:
            print(f"Skipping k={k} due to error: {e}")
            return 1e6  # Return a large value to penalize the failed set of hyperparameters

    result = gp_minimize(objective_k_mknn, [(k_min, k_max)], n_calls=20, n_random_starts=10, random_state=42)

    if result.fun < 1e6:
        k_mknn = result.x[0]
        print(f"Best k for mknn: {k_mknn}")
        return result
    else:
        print("No valid k for mknn found.")
        return None

In [ ]:
# Call the optimization functions
laplacian_methods = ["sym", "rw", "ad"]
number_of_clusters = 2

# Optimize local_sigma for "full" graph
result_local_sigma = optimize_local_sigma(data_cleaned, true_labels, laplacian_methods, number_of_clusters)
best_local_sigma = result_local_sigma.x[0]

# Optimize epsilon for "eps" graph
result_epsilon = optimize_epsilon(data_cleaned, true_labels, laplacian_methods, number_of_clusters)
best_epsilon = round(result_epsilon.x[0], 3)

# Optimize k for "knn" graph
result_k_knn = optimize_k_knn(data_cleaned, true_labels, laplacian_methods, number_of_clusters)
best_k_knn = result_k_knn.x[0]

# Optimize k for "mknn" graph
result_k_mknn = optimize_k_mknn(data_cleaned, true_labels, laplacian_methods, number_of_clusters)
best_k_mknn = result_k_mknn.x[0]

100%|██████████████████████████████████████| 1716/1716 [00:03<00:00, 442.32it/s]


In [ ]:
similarity_graphs = ["full", "eps", "knn", "mknn"]
laplacian_methods = ["sym", "rw","ad"]
number_of_clusters = 3
# best_local_sigma = 11
# best_epsilon = 5
# best_k_knn = 29
# best_k_mknn = 28

silhouette_scores = []
adjusted_rand_scores = []
clusters = []
sim_graph = []
laplacian = []
cluster_labels = []
hyperparameters = []

In [ ]:
for graph in similarity_graphs:

    for laplace in laplacian_methods:
        metrics = spectral_clustering(data_cleaned, true_labels, graph, laplace, number_of_clusters, best_local_sigma, best_epsilon, best_k_knn, best_k_mknn)
        # metrics, adj_file, lap_file, X_file = spectral_clustering(data_df, true_labels, graph, laplace, number_of_clusters, best_local_sigma, best_epsilon, best_k_knn, best_k_mknn)

        for si, ar, cl, l in metrics:
            sim_graph.append(graph)
            laplacian.append(laplace)
            silhouette_scores.append(si)
            adjusted_rand_scores.append(ar)
            clusters.append(cl)
            cluster_labels.append(l)
            # Append consolidated hyperparameters for each similarity graph type
            if graph == "full":
                hyperparameters.append(f"local_sigma={best_local_sigma}")
            elif graph == "eps":
                hyperparameters.append(f"epsilon={best_epsilon}")
            elif graph == "knn":
                hyperparameters.append(f"k_nn={best_k_knn}")
            elif graph == "mknn":
                hyperparameters.append(f"k_mknn={best_k_mknn}")
            else:
                hyperparameters.append("None")

In [ ]:
experiment_madelon = pd.DataFrame(list(zip(sim_graph,laplacian,silhouette_scores,adjusted_rand_scores,clusters, hyperparameters, cluster_labels)),
             columns= ["graph","laplacian", "silhouette", "adjusted_rand","number_of_clusters","hyperparameters", "cluster_labels"])
experiment_madelon["graph_laplacian"] = experiment_iris["graph"] + "_" + experiment_iris["laplacian"]
experiment_madelon

In [ ]:
# Save the DataFrame to a CSV file
experiment_madelon.to_csv('experiment_madelon_66.csv', index=False)

# File is now saved in the current working directory
print("CSV file saved as 'experiment_madelon_66.csv'")